# Hyperparameters and Model validation

In the previous section, we saw the basic recipe of applying a supervised machine learning model:

1. Choose a class of model 
2. Choose model hyperparameters
3. Fit the model to the training data
4. Use the model to predict labels for new data 

The first two pieces of this - the choice of model and choice of hyperparameters are perhaps the most
important part of using these tools and techniques effectively. In order to make an informed choice, we need a way to *validate* that our model and our hyperparameters are a good fit to the data. While this may sound simple, there are some pitfalls that you must avoid to do this effectively.

## Thinking about Model Validation

In principle, model validation is very simple: after choosing a model and its hyperparameters, we can estimate how effective it is by applying it to some of the training data and comparing the prediction to the known value.

The following sections first show a naive approach to model validation and why it fails, before exploring the use of houldout sets and cross-validation for more robust model evaluation.